Many words: w_1, w_2, ... w_i

X_i: the message contains w_i

P(X_i|S): prob. a spam message contains w_i

P(X_i|~S): prob. a nonspam message contains w_i

"The key to Naive Bayes is making the (big) assumption that the presences (or absences) of each word are independent of one another, conditional on a message being spam or not."

"The Naive Bayes assumption allows us to compute each of the probabilities on the right simply by multiplying together the individual probability estimates for each vocabulary word."

P(S|X=x) = P(X=x|S)/\[P(X=x|S) + P(X=x|~S)\]

"In practice, you usually want to avoid multiplying lots of probabilities together, to avoid a problem called underflow, in which computers don’t deal well with floating-point numbers that are too close to zero. Recalling from algebra that *log ab = log a + log b* and that *exp log x = x*, we usually compute p 1 * ⋯ * p n as the equivalent (but floating-point-friendlier):"

exp (log p1 + ⋯ + log pn)

"The only challenge left is coming up with estimates for P(X_i|S) and P(X_i|~S), the probabilities that a spam message (or nonspam message) contains the word w_i . If we have a fair number of 'training' messages labeled as spam and not-spam, an obvious
first try is to estimate P(X_i|S) simply as the fraction of spam messages containing word w_i."

* This causes a big problem, though. Imagine that in our training set the vocabulary word “data” only occurs in nonspam messages. Then we’d estimate P(“data”|S) = 0. The result is that our Naive Bayes classifier would always assign spam probability 0 to any message containing the word “data,” even a message like “data on cheap viagra and authentic rolex watches.” To avoid this problem, we usually use some kind of smoothing.

"In particular, we’ll choose a pseudocount — *k* — and estimate the probability of seeing the ith word in a spam as:"

P(X_i|S) = (k + number of spams containing w_i)/(2k + number of spams)

* "Similarly for P(X_i|~S). That is, when computing the spam probabilities for the ith word, we assume we also saw k additional spams containing the word and k additional spams not containing the word."

### Implementation

In [2]:
def tokenize(message):
    # convert to lowercase
    message = message.lower()
    # extract the words
    all_words = re.findall("[a-z0-9']+", message)
    # remove duplicates
    return set(all_words)

"Our second function will count the words in a labeled training set of messages. We’ll have it return a dictionary whose keys are words, and whose values are two-element lists \[spam_count, non_spam_count\] corresponding to how many times we saw that
word in both spam and nonspam messages:"

In [4]:
def count_words(training_set):
    """
    training set consists of pairs (message, is_spam)
    """
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1

    return counts

In [5]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """
    turn the word_counts into a list of triplets
    w, p(w | spam) and p(w | ~spam)
    """
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k))
            for w, (spam, non_spam) in counts.iteritems()]

In [6]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    # iterate through each word in our vocabulary
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        # if *word* appears in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        # if *word* doesn't appear in the message
        # add the log probability of _not_ seeing it
        # which is log(1 - probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [7]:
class NaiveBayesClassifier:
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def train(self, training_set):
        # count spam and non-spam messages
        num_spams = len([is_spam 
                         for message, is_spam in training_set 
                         if is_spam])
        num_non_spams = len(training_set) - num_spams
        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,num_spams,
                                             num_non_spams,self.k)

    def classify(self, message):
        return spam_probability(self.word_probs, message)

"How could we get better performance?
* One obvious way would be to get more data to train on."

"There are a number of ways to improve the model as well."
* Our classifier takes into account every word that appears in the training set, even words that appear only once. Modify the classifier to accept an optional *min_count* threshhold and ignore tokens that don’t appear at least that many times.
* The tokenizer has no notion of similar words (e.g., “cheap” and “cheapest”). Modify the classifier to take an optional stemmer function that converts words to equivalence classes of words. For example, a really simple stemmer function might be:

In [8]:
def drop_final_s(word):
    return re.sub("s$", "", word)

* Creating a good stemmer function is hard. People frequently use the **Porter Stemmer**.

* Although our features are all of the form “message contains word w_i ,” there’s no reason why this has to be the case. In our implementation, we could add extra features like “message contains a number” by creating phony tokens like contains:number and modifying the tokenizer to emit them when appropriate.

"*scikit-learn* contains a BernoulliNB model that implements the same Naive Bayes algorithm we implemented here, as well as other variations on the model.